In [ ]:
pip install pyspark


In [ ]:
pip install alpha_vantage

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
import os
import pandas as pd
from functools import reduce
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print(cwd)

In [1]:
## Capture SNP500 companies data & put symbols in a list ##

import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
first_table = payload[0]
second_table = payload[1]
third_table = payload[2]

df = third_table
symbols = df['Symbol'].values.tolist()


In [ ]:
## Hit AlphaVantage API for 1 minute interval prices and insert into csvs ##

import csv
import requests
import time

def price_wrapper(tickers):

    file_list =[]    
    for x in tickers:
        CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+x+'&interval=1min&slice=year1month1&apikey=VOTCB8UWEES6J5NP'
        
        with requests.Session() as s:
            download = s.get(CSV_URL)
            decoded_content = download.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            my_list = list(cr)
            
            ##append column with symbol
            my_list[0].insert(0,'symbol')
            for row in my_list[1:(len(my_list)-1)]:
                row.insert(0,x)
            
            file = "/YXRwLW5vdGVib29rLXN0cHluZS1zZXNzaW9uLWpydWw=/stpyne/symbols_prices_new/"+x+".csv"
            
            file_list.append(file)
            
            with open(file, 'w') as csvfile: 
                csvwriter = csv.writer(csvfile) 
                csvwriter.writerow(my_list[0])
                for row in my_list[1:(len(my_list)-1)]:
                    csvwriter.writerow(row)
        
        #use sleep to ensure do not max out 5 API calls per minute
        time.sleep(60)

          


In [2]:
import os
import tweepy as tw
import pandas as pd
##Twitter API Auth Setup

consumer_key= '6jvo5IFoAAoLLznFVJm6xb1N1'
consumer_secret= 'UqAipjwJUPrOVE0tLOHoqPFaxA4ZKpWRvENJrlknK2AJKGsr5u'
access_token= '1422620267059183618-bOU2AlKVbb8UU9HeBg1g87YaWcyCRj'
access_token_secret= 'fAhSJD3l0AYt4FKvJU7V9zYkWOfcg7RNksVkdqcCAZOAT'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)



In [ ]:
## Query Twitter API 


import csv
import datetime
import time
countlist = []
tweetcount= 0
diff=0
with open ('symbols_tweets_upd1.csv', 'a+', newline='',encoding='utf-8') as csvFile:
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow(['symbol','tweet_id','text','created_at','retweets','likes','geo','place','coordinates','location'])                     
    
    
    for symbol in symbols:
        if len(countlist) > 15000 and diff<500 and diff <900:
            print('max api reached. total tweets:' + str(tweetcount) +" -- seconds elapsed:"+str(diff)+"....sleeping")
            time.sleep(700)
            diff = 0
            countlist = []
        else:
            print('total tweets queried so far:' + str(tweetcount) +" -- seconds elapsed:"+str(diff)+"...running")
        
            
        tic = time.perf_counter()
        print(symbol+' start')
        
        for tweet in tw.Cursor(api.search,
                           q = "$" + symbol,
                           until = "2021-08-7",
                           lang = "en",
                              ).items(4000):
           
    # Write a row to the CSV file. I use encode UTF-8
            tweets_encoded = tweet.text.encode('utf-8')
            tweets_decoded = tweets_encoded.decode('utf-8')
                        
            csvWriter.writerow([symbol, tweet.id, tweets_decoded, tweet.created_at,tweet.retweet_count, tweet.favorite_count, tweet.geo, tweet.place.name if tweet.place else None, tweet.coordinates, tweet._json["user"]["location"]])
            countlist.append(tweet.id)
        print(symbol + " finish")
        
        toc = time.perf_counter()
        diff = diff + (toc - tic)
        tweetcount = len(countlist)
        
        

In [9]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import length
from pyspark.sql.functions import col

sc = SparkContext.getOrCreate();
spark = SparkSession(sc)    
df = spark.read.load("symbols_tweets_mod.csv",
                     format="csv", sep=",", header="true",multiline="true")

df_upd = df.where(length(col("symbol")) <= 6)


In [6]:
##getting weird error "likes" is not a numeric column. come back to this....

from pyspark.sql.types import IntegerType
df_upd = df_upd.withColumn("symbol", df_upd["symbol"].cast(IntegerType()))
df_upd.schema

#df_upd.groupBy("symbol").sum("likes").show(truncate=False)



StructType(List(StructField(symbol,IntegerType,true),StructField(tweet_id,StringType,true),StructField(text,StringType,true),StructField(created_at,StringType,true),StructField(retweets,StringType,true),StructField(likes,StringType,true),StructField(geo,StringType,true),StructField(place,StringType,true),StructField(coordinates,StringType,true),StructField(location,StringType,true)))

In [11]:

df_upd.groupBy(‘symbol’).count().show()



SyntaxError: invalid character in identifier (<ipython-input-11-08f1dda56af9>, line 1)

In [ ]:
df_upd.toPandas()['k'].unique()
